# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,huarmey,-10.0681,-78.1522,20.63,84,43,3.77,PE,1714866817
1,1,waddan,29.1614,16.1390,19.62,39,0,4.14,LY,1714866956
2,2,edinburgh of the seven seas,-37.0676,-12.3116,14.31,76,46,15.31,SH,1714866775
3,3,bredasdorp,-34.5322,20.0403,12.71,81,8,1.86,ZA,1714866956
4,4,wynyard,-41.0000,145.7167,14.70,54,2,1.07,AU,1714866956


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
city_data_df_clean = city_data_df.dropna()
city_data_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 545 entries, 0 to 548
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     545 non-null    int64  
 1   City        545 non-null    object 
 2   Lat         545 non-null    float64
 3   Lng         545 non-null    float64
 4   Max Temp    545 non-null    float64
 5   Humidity    545 non-null    int64  
 6   Cloudiness  545 non-null    int64  
 7   Wind Speed  545 non-null    float64
 8   Country     545 non-null    object 
 9   Date        545 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 46.8+ KB


In [5]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.95,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_ideal_1 = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27), :]
city_data_df_ideal_2 = city_data_df_ideal_1.loc[city_data_df_ideal_1["Wind Speed"] < 4.5, :]
city_data_df_ideal_final = city_data_df_ideal_2.loc[city_data_df_ideal_2["Cloudiness"] == 0, :]

# Drop any rows with null values
city_data_df_ideal_final_clean = city_data_df_ideal_final.dropna()

# Display sample data
city_data_df_ideal_final_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,morondava,-20.2833,44.2833,23.54,63,0,4.05,MG,1714866972
101,101,gebeit,21.0667,36.3167,22.23,26,0,1.52,SD,1714866974
124,124,ad dilam,23.9915,47.1627,26.23,28,0,4.11,SA,1714866979
178,178,al wafrah,28.6392,47.9306,25.26,43,0,3.22,KW,1714866990
191,191,bushehr,28.9684,50.8385,25.05,78,0,4.12,IR,1714866992
221,221,pisco,-13.7000,-76.2167,21.03,83,0,3.09,PE,1714866998
246,246,cachoeiro de itapemirim,-20.8489,-41.1128,21.30,92,0,0.56,BR,1714867003
253,253,goldsboro,35.3849,-77.9928,25.96,60,0,3.09,US,1714867005
255,255,turabah,21.2141,41.6331,23.93,33,0,0.60,SA,1714867005
306,306,tokar,18.4254,37.7290,25.71,57,0,3.06,SD,1714867014


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = city_data_df_ideal_final_clean[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
92,morondava,MG,-20.2833,44.2833,63,
101,gebeit,SD,21.0667,36.3167,26,
124,ad dilam,SA,23.9915,47.1627,28,
178,al wafrah,KW,28.6392,47.9306,43,
191,bushehr,IR,28.9684,50.8385,78,
221,pisco,PE,-13.7000,-76.2167,83,
246,cachoeiro de itapemirim,BR,-20.8489,-41.1128,92,
253,goldsboro,US,35.3849,-77.9928,60,
255,turabah,SA,21.2141,41.6331,33,
306,tokar,SD,18.4254,37.7290,57,


In [21]:
# Set parameters to search for a hotel
Lng = hotel_df["Lng"]
Lat = hotel_df["Lat"]

radius = 10000
limit = 20
filters = f"circle:{Lng},{Lat},{radius}"
bias = f"proximity:{Lng},{Lat}"

params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    city = hotel_df.loc[index, "City"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    params["categories"] = "accommodation"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    #constructed_url = base_url + "?" + "&".join([f"{key}={value}" for key, value in params.items()])
    #print(f"Constructed URL: {constructed_url}")
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
morondava - nearest hotel: Hotel Sharon Menabe
gebeit - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
al wafrah - nearest hotel: No hotel found
bushehr - nearest hotel: هتل رستوران سیراف
pisco - nearest hotel: No hotel found
cachoeiro de itapemirim - nearest hotel: Bristol Easy Hotel - Cachoeiro
goldsboro - nearest hotel: Economy Inn
turabah - nearest hotel: No hotel found
tokar - nearest hotel: No hotel found
jesup - nearest hotel: A-One Motel and Apartments
namibe - nearest hotel: Pensão Delta do Deserto
lincheng - nearest hotel: 小城之春假日酒店
ha'il - nearest hotel: فندق كلاسيك
natal - nearest hotel: Natal Palace Hotel
tanumah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
92,morondava,MG,-20.2833,44.2833,63,Hotel Sharon Menabe
101,gebeit,SD,21.0667,36.3167,26,No hotel found
124,ad dilam,SA,23.9915,47.1627,28,No hotel found
178,al wafrah,KW,28.6392,47.9306,43,No hotel found
191,bushehr,IR,28.9684,50.8385,78,هتل رستوران سیراف
221,pisco,PE,-13.7000,-76.2167,83,No hotel found
246,cachoeiro de itapemirim,BR,-20.8489,-41.1128,92,Bristol Easy Hotel - Cachoeiro
253,goldsboro,US,35.3849,-77.9928,60,Economy Inn
255,turabah,SA,21.2141,41.6331,33,No hotel found
306,tokar,SD,18.4254,37.7290,57,No hotel found


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
Lng = hotel_df["Lng"]
Lat = hotel_df["Lat"]

radius = 10000
limit = 20
filters = f"circle:{Lng},{Lat},{radius}"
bias = f"proximity:{Lng},{Lat}"

params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    city = hotel_df.loc[index, "City"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    params["categories"] = "accommodation"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    #constructed_url = base_url + "?" + "&".join([f"{key}={value}" for key, value in params.items()])
    #print(f"Constructed URL: {constructed_url}")
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
morondava - nearest hotel: Hotel Sharon Menabe
gebeit - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
al wafrah - nearest hotel: No hotel found
bushehr - nearest hotel: هتل رستوران سیراف
pisco - nearest hotel: No hotel found
cachoeiro de itapemirim - nearest hotel: Bristol Easy Hotel - Cachoeiro
goldsboro - nearest hotel: Economy Inn
turabah - nearest hotel: No hotel found
tokar - nearest hotel: No hotel found
jesup - nearest hotel: A-One Motel and Apartments
namibe - nearest hotel: Pensão Delta do Deserto
lincheng - nearest hotel: 小城之春假日酒店
ha'il - nearest hotel: فندق كلاسيك
natal - nearest hotel: Natal Palace Hotel
tanumah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
92,morondava,MG,-20.2833,44.2833,63,Hotel Sharon Menabe
101,gebeit,SD,21.0667,36.3167,26,No hotel found
124,ad dilam,SA,23.9915,47.1627,28,No hotel found
178,al wafrah,KW,28.6392,47.9306,43,No hotel found
191,bushehr,IR,28.9684,50.8385,78,هتل رستوران سیراف
221,pisco,PE,-13.7000,-76.2167,83,No hotel found
246,cachoeiro de itapemirim,BR,-20.8489,-41.1128,92,Bristol Easy Hotel - Cachoeiro
253,goldsboro,US,35.3849,-77.9928,60,Economy Inn
255,turabah,SA,21.2141,41.6331,33,No hotel found
306,tokar,SD,18.4254,37.7290,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
# Configure the map
# Configure the map plot_4
map_plot_H = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1.50,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map plot
map_plot_H

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)